In [ ]:
# Start H2O-3 runtime.

import h2o
h2o.init(strict_version_check=False)

In [ ]:
# Configure K-Means algorithm and train a model.

from h2o.estimators import H2OKMeansEstimator

# Import the iris dataset into H2O:
iris = h2o.import_file("http://h2o-public-test-data.s3.amazonaws.com/smalldata/iris/iris_wheader.csv")

# Set the predictors:
predictors = ["sepal_len", "sepal_wid", "petal_len", "petal_wid"]

# Split the dataset into a train and valid set:
train, valid = iris.split_frame(ratios=[.8], seed=1234)

# Build and train the model:
kmeans = H2OKMeansEstimator(k=10,
                            estimate_k=True,
                            standardize=False,
                            seed=1234)
kmeans.train(x=predictors,
                  training_frame=train,
                  validation_frame=valid)

In [ ]:
# Log the model to an MLFlow reqistry.

import mlflow
import h2o_mlflow_flavor
mlflow.set_tracking_uri("http://127.0.0.1:8080")

with mlflow.start_run(run_name="iris") as run:
    mlflow.log_params(h2o_mlflow_flavor.get_params(kmeans)) # Log training parameters of the model (optional).
    mlflow.log_metrics(h2o_mlflow_flavor.get_metrics(kmeans)) # Log performance matrics of the model (optional).
    input_example = h2o_mlflow_flavor.get_input_example(kmeans) # Extract input example from training dataset (optional)
    h2o_mlflow_flavor.log_model(kmeans, "iris", input_example=input_example,
                                model_type="POJO") # Specify whether the output model should be MOJO or POJO. (MOJO is default)
                                

In [ ]:
# Load model from the MLFlow registry and score with the model.

import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:8080")

logged_model = 'runs:/9a42265cf0ef484c905b02afb8fe6246/iris' # Specify correct id of your run.

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
data = pd.read_csv("http://h2o-public-test-data.s3.amazonaws.com/smalldata/iris/iris_wheader.csv")
loaded_model.predict(data)